In [1]:
import torch

device = torch.device('cuda')

In [2]:
from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": 'data/merged_delete_later.txt'})

Using custom data configuration default-d8421625fbe02c62
Reusing dataset text (C:\Users\anzor\.cache\huggingface\datasets\text\default-d8421625fbe02c62\0.0.0\e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
eval_dataset = load_dataset('opus100', 'en-ru', split='train')

Reusing dataset opus100 (C:\Users\anzor\.cache\huggingface\datasets\opus100\en-ru\0.0.0\256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)


In [79]:
from transformers import MT5Tokenizer

input_lang = 'ru'
output_lang = 'kbd'

input_tokenizer = MT5Tokenizer.from_pretrained('tokenizers/'+ input_lang + '/spiece.model', extra_ids=0)
output_tokenizer = MT5Tokenizer.from_pretrained('tokenizers/'+ output_lang +'/spiece.model', extra_ids=0)

In [114]:
from transformers import MT5ForConditionalGeneration

model = MT5ForConditionalGeneration.from_pretrained('models/epoch_29')

model.to(device)

# model.config.vocab_size = len(tokenizer.get_vocab())  # remove later, bug fixing

MT5ForConditionalGeneration(
  (shared): Embedding(32497, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32497, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo)

In [121]:
# from bleu_eval import sacre_bleu_score
import random
import re

# input_text, target_text = dataset['train'][random.randint(0, len(dataset['train']) - 1)]['text'].split('😀')

# target_text = 'test'
# eval_lines = re.split('(?<=[.!?]) +', eval_dataset['train'][random.randint(0, len(eval_dataset['train']) - 1)]['text'])
# input_text = eval_lines[random.randint(0, len(eval_lines)-1)]
eval_lines = eval_dataset[random.randint(0, len(eval_dataset) - 1)]['translation']
input_text =eval_lines['ru']
# target_text =eval_lines['en']


input_text = "твой дом"
# target_text = "When I looked again I noticed something shiny."
encodings = input_tokenizer(
    # "<s> " +
    input_text,
    # + "</s>",
    padding='longest',
    max_length=512,
    truncation=True,
    return_tensors="pt")

input_ids = encodings.input_ids

model.cpu()

n_beams = 5
# for i in range(1, n_beams + 1):
output_tokens = model.generate(input_ids,
                               num_beams=n_beams,
                               num_return_sequences=n_beams,
                               # output_attentions=True,
                               # output_scores = True,
                               # repetition_penalty = 5.5,
                               length_penalty=0.1,
                               # do_sample=True,
                               # temperature=0.1,
                               # bos_token_id = 2,
                               # output_scores=True,
                               # max_length=128
                               )

# output_tokens[output_tokens==0]=1 # replace <unk> with <s> (— bos token)

# print(str(i) + ' ' + input_text + '  ->  ' + tokenizer.decode(output_tokens[0], skip_special_tokens=True))
# print('n_beams: ' + str(i))
print('n_beams: ' + str(n_beams))
print('input: ' + input_text + '\n')
for token_set in output_tokens:
    prediction = output_tokenizer.decode(token_set, skip_special_tokens=True)
    print("output: {}".format(prediction))
    # print('BLEU score: ' + str(sacre_bleu_score(predictions=prediction, references=target_text)['score']) + '\n')
# print('reference: ' + target_text)

n_beams: 5
input: твой дом

output: унэ
output: унэ унэ
output: унэ лъагэр
output: унэ (нэ
output: унэ (нэр


In [7]:
output_tokenizer.tokenize('джабэм ехащ')

['▁', 'дж', 'аб', 'эм', '▁е', 'х', 'ащ']

In [8]:
# import regex as re
#
# def extract_pairs(text):
#     pattern = r'<i>(\X[^i]+)<\/i>(?:[^«<])*«(\X[^»]+)»'
#     return re.findall(pattern, text)
#
# # save the pairs line by line to a txt file in the following format: x😀y
# def save_pairs(pairs, filename):
#     with open(filename, 'w', encoding='utf-8') as f:
#         for pair in pairs:
#             f.write(pair[0].replace('\n', ' ') + '😀' + pair[1].replace('\n', ' ') + '\n')
#
# def main():
#     with open('data/1.html', 'r', encoding='utf-8') as f:
#         text = f.read()
#     pairs = extract_pairs(text)
#     save_pairs(pairs, 'kbd-ru.txt')

In [9]:
# main()

In [10]:
# ### create a russian text dump for tokenization
# !pip install datasets
# from datasets import load_dataset
# dataset = load_dataset('mc4', 'ru', streaming=True)
# from tqdm.notebook import tqdm
#
# iterator = iter(dataset['train'])
# with open('data/for_tokenizer/tokenizer_data_ru.txt', 'w', encoding="utf-8") as f:
#     for i in tqdm(range(50_000)):
#         text = next(iterator)['text']
#         for line in text.splitlines():
#             f.write(line+'\n')